#Royal Cybers: End-to-End Machine Learning Pipeline for Personalized recommendations in Databricks 

## Load Dataset

In [0]:
# Load Dataset from S3 Bucket
cosmetic_store_data_path = "s3://e-commerce-pipeline-dataset/Cosmetic Store Website Data.csv"
reviews_data_path = "s3://e-commerce-pipeline-dataset/nyka_top_brands_cosmetics_product_reviews.csv"
product_mapping_path = "s3://e-commerce-pipeline-dataset/unique_product_id_pairings.csv"

cosmetic_df = spark.read.csv(cosmetic_store_data_path, header=True, inferSchema=True)
reviews_df = spark.read.csv(reviews_data_path, header=True, inferSchema=True)
mapping_df = spark.read.csv(product_mapping_path, header=True, inferSchema=True)

In [0]:
cosmetic_df.show(5)

+-------------------+----------+----------+-------------------+-------------+--------+-----+---------+--------------------+
|         event_time|event_type|product_id|        category_id|category_code|   brand|price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+-------------+--------+-----+---------+--------------------+
|2020-01-01 00:00:00|      view|   5809910|1602943681873052386|         NULL| grattol| 5.24|595414620|4adb70bb-edbd-498...|
|2020-01-01 00:00:09|      view|   5812943|1487580012121948301|         NULL|kinetics| 3.97|595414640|c8c5205d-be43-4f1...|
|2020-01-01 00:00:19|      view|   5798924|1783999068867920626|         NULL|  zinger| 3.97|595412617|46a5010f-bd69-4fb...|
|2020-01-01 00:00:24|      view|   5793052|1487580005754995573|         NULL|    NULL| 4.92|420652863|546f6af3-a517-475...|
|2020-01-01 00:00:25|      view|   5899926|2115334439910245200|         NULL|    NULL| 3.92|484071203|cff70ddf-529e-4b0...|
+-------

In [0]:
reviews_df.show(5)

+----------+----------+---------+--------------------+--------------------+--------------+-------------------+-------------+----------+--------+--------------+--------------------+----+-----+--------------+--------------------+------------+--------------------+
|product_id|brand_name|review_id|        review_title|         review_text|        author|        review_date|review_rating|is_a_buyer|pro_user|  review_label|       product_title| mrp|price|product_rating|product_rating_count|product_tags|         product_url|
+----------+----------+---------+--------------------+--------------------+--------------+-------------------+-------------+----------+--------+--------------+--------------------+----+-----+--------------+--------------------+------------+--------------------+
|    781070|      Olay| 16752142|Worth buying 50g one|Works as it claim...| Ashton Dsouza|2021-01-23 15:17:18|          5.0|      True|   False|Verified Buyer|Olay Ultra Lightw...|1999| 1599|           4.1|        

In [0]:
mapping_df.show(5)

+-----------------+------------------+
|product_id_events|product_id_reviews|
+-----------------+------------------+
|            11719|            209281|
|            15161|           2645330|
|            16686|           3445855|
|            27658|           4749403|
|            34768|            995305|
+-----------------+------------------+
only showing top 5 rows



%md
## Initial Data Cleaning

In [0]:
# Drop Duplicates
cosmetic_df = cosmetic_df.dropDuplicates()
reviews_df = reviews_df.dropDuplicates()
mapping_df = mapping_df.dropDuplicates()

In [0]:
# Standardizing column names
cosmetic_df = cosmetic_df.withColumnRenamed("product_id", "cosmeticProductId") \
                         .withColumnRenamed("price", "cosmetic_price") 

reviews_df = reviews_df.withColumnRenamed("product_id", "reviewProductId") \
                       .withColumnRenamed("price", "review_price")

mapping_df = mapping_df.withColumnRenamed("product_id_events", "cosmeticProductId") \
                       .withColumnRenamed("product_id_reviews", "reviewProductId")

In [0]:
# Drop unnecessary columns and fill missing values
cosmetic_df = cosmetic_df.filter(cosmetic_df['cosmetic_price'] > 0)

mapping_df = mapping_df.na.drop()

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

In [0]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = text.replace('\n', ' ')  # Remove new lines
        text = text.replace('\t', ' ')  # Remove tabs
        text = text.replace('\r', ' ')  # Remove returns
        text = re.sub(r'[^a-z\s]', '', text)  # Remove non-alphanumeric characters except spaces
        words = text.split()  # Tokenize the text
        words = [word for word in words if word not in stop_words]  # Remove stopwords
        text = ' '.join(words)  # Combine words back into text
        return text.strip()
    return text  # Return original value if not a string (e.g., NaN)

# Register the clean_text function as a UDF
clean_text_udf = udf(clean_text, StringType())

# Drop unnecessary columns and handle missing values
reviews_df = reviews_df.drop("product_tags")
reviews_df = reviews_df.fillna({
    'review_text': 'No review', 
    'brand_name': 'Unknown', 
    'review_label': 'No Label',
    'product_title': 'Unknown Title'
})

# Apply the UDF to clean the review_title and review_text columns
reviews_df = reviews_df.withColumn("review_title_clean", clean_text_udf(reviews_df["review_title"]))
reviews_df = reviews_df.withColumn("review_text_clean", clean_text_udf(reviews_df["review_text"]))

# Show the results
reviews_df.select("review_title", "review_title_clean", "review_text", "review_text_clean").show(5)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


+------------------+------------------+--------------------+--------------------+
|      review_title|review_title_clean|         review_text|   review_text_clean|
+------------------+------------------+--------------------+--------------------+
|Invisible & Smooth|  invisible smooth|Wore these when I...|wore massive brea...|
|              Nice|              nice|Works as it claim...|works claims smal...|
|             Great|             great|It Won't let your...|wont let pimple g...|
|          Easy fix|          easy fix|This is one easy ...|one easy fix redu...|
|        Give a try|          give try|This is my first ...|first purchase tr...|
+------------------+------------------+--------------------+--------------------+
only showing top 5 rows



In [0]:
# Filtering our multi classificaiton 
valid_events = ["view", "cart", "remove_from_cart", "purchase"]
cosmetic_df = cosmetic_df.filter(cosmetic_df.event_type.isin(valid_events))

cosmetic_df.groupBy("event_type").count().show()

+----------------+-------+
|      event_type|  count|
+----------------+-------+
|        purchase| 263570|
|            view|2032076|
|            cart|1126995|
|remove_from_cart| 609461|
+----------------+-------+



In [0]:
cosmetic_df.show(5)

+-------------------+----------+-----------------+-------------------+-------------+--------+--------------+---------+--------------------+
|         event_time|event_type|cosmeticProductId|        category_id|category_code|   brand|cosmetic_price|  user_id|        user_session|
+-------------------+----------+-----------------+-------------------+-------------+--------+--------------+---------+--------------------+
|2020-01-01 00:00:19|      view|          5798924|1783999068867920626|         NULL|  zinger|          3.97|595412617|46a5010f-bd69-4fb...|
|2020-01-01 00:00:25|      view|          5899926|2115334439910245200|         NULL|    NULL|          3.92|484071203|cff70ddf-529e-4b0...|
|2020-01-01 00:00:24|      view|          5793052|1487580005754995573|         NULL|    NULL|          4.92|420652863|546f6af3-a517-475...|
|2020-01-01 00:00:09|      view|          5812943|1487580012121948301|         NULL|kinetics|          3.97|595414640|c8c5205d-be43-4f1...|
|2020-01-01 00:00:00

In [0]:
reviews_df.show(5)

+---------------+--------------+---------+------------------+--------------------+-----------------+-------------------+-------------+----------+--------+--------------+--------------------+---+------------+--------------+--------------------+--------------------+------------------+--------------------+
|reviewProductId|    brand_name|review_id|      review_title|         review_text|           author|        review_date|review_rating|is_a_buyer|pro_user|  review_label|       product_title|mrp|review_price|product_rating|product_rating_count|         product_url|review_title_clean|   review_text_clean|
+---------------+--------------+---------+------------------+--------------------+-----------------+-------------------+-------------+----------+--------+--------------+--------------------+---+------------+--------------+--------------------+--------------------+------------------+--------------------+
|         785729|Nykaa Naturals| 27389259|Invisible & Smooth|Wore these when I...|   

In [0]:
mapping_df.show(5)

+-----------------+---------------+
|cosmeticProductId|reviewProductId|
+-----------------+---------------+
|            34768|         995305|
|            15161|        2645330|
|            16686|        3445855|
|            19162|        3494214|
|            11719|         209281|
+-----------------+---------------+
only showing top 5 rows



## Manage Delta Tables

In [0]:
# dbutils.fs.rm("dbfs:/mnt/delta/cosmetic_store_data", True)  # True for recursive delete
# dbutils.fs.rm("dbfs:/mnt/delta/product_reviews", True)  # True for recursive delete
# dbutils.fs.rm("dbfs:/mnt/delta/product_mapping", True)  # True for recursive delete

In [0]:
cosmetic_delta_path = "/mnt/delta/cosmetic_store_data"
reviews_delta_path = "/mnt/delta/product_reviews"
mapping_delta_path = "/mnt/delta/product_mapping"

In [0]:
# Convert each cleaned DataFrame to Delta format
cosmetic_df.write.format("delta").mode("overwrite").save(cosmetic_delta_path)
reviews_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(reviews_delta_path)
mapping_df.write.format("delta").mode("overwrite").save(mapping_delta_path)


In [0]:
# Load Delta tables directly from paths
cosmetic_df = spark.read.format("delta").load(cosmetic_delta_path)
reviews_df = spark.read.format("delta").load(reviews_delta_path)
mapping_df = spark.read.format("delta").load(mapping_delta_path)


## Feature Engineering

### Unstucutred data 

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag, word_tokenize
import nltk

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import StringType

reviews_df = reviews_df.withColumn("review_title_clean", col("review_title_clean").cast(StringType()))
reviews_df = reviews_df.withColumn("review_text_clean", col("review_text_clean").cast(StringType()))


In [0]:
reviews_df = reviews_df.fillna({'review_title_clean': 'No title', 'review_text_clean': 'No review'})

In [0]:
!pip install spacy
!python -m spacy download en_core_web_sm
import spacy
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 81.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [0]:
nlp = spacy.load("en_core_web_sm")

# Define the stemming and lemmatization functions
def process_text_spacy(text):
    try:
        if isinstance(text, str) and text:
            doc = nlp(text)
            lemmatized = " ".join([token.lemma_ for token in doc])  # Lemmatization using SpaCy
            stemmed = " ".join([token.text[:3] for token in doc])   # Simulated "stemming" by truncating words (example)
            return stemmed, lemmatized
        else:
            return "", ""  # Handle invalid input gracefully
    except Exception as e:
        print(f"Error processing text: {text} | Error: {e}")
        return "", ""

# Register UDFs for stemming and lemmatization
stem_udf = udf(lambda text: process_text_spacy(text)[0], StringType())
lemmatize_udf = udf(lambda text: process_text_spacy(text)[1], StringType())

# Fill missing values for relevant columns
reviews_df = reviews_df.fillna({
    "review_title_clean": "No title",
    "review_text_clean": "No review"
})

# Apply UDFs to DataFrame columns
reviews_df = reviews_df.withColumn("stemmed_title", stem_udf(reviews_df["review_title_clean"]))
reviews_df = reviews_df.withColumn("lemmatized_title", lemmatize_udf(reviews_df["review_title_clean"]))
reviews_df = reviews_df.withColumn("stemmed_text", stem_udf(reviews_df["review_text_clean"]))
reviews_df = reviews_df.withColumn("lemmatized_text", lemmatize_udf(reviews_df["review_text_clean"]))

# Show results for titles
reviews_df.select("review_title_clean", "stemmed_title", "lemmatized_title").show(5)

# Show results for text
reviews_df.select("review_text_clean", "stemmed_text", "lemmatized_text").show(5)

+------------------+-------------+----------------+
|review_title_clean|stemmed_title|lemmatized_title|
+------------------+-------------+----------------+
|        best cream|      bes cre|      good cream|
|          good one|      goo one|        good one|
|       rd purchase|       rd pur|     rd purchase|
|        nykaa skin|      nyk ski|      nykaa skin|
|     great product|      gre pro|   great product|
+------------------+-------------+----------------+
only showing top 5 rows

+--------------------+--------------------+--------------------+
|   review_text_clean|        stemmed_text|     lemmatized_text|
+--------------------+--------------------+--------------------+
|literally best cr...| lit bes cre ant agi|literally good cr...|
|                    |                    |                    |
|rd purchase good ...|rd pur goo tim bu...|rd purchase good ...|
|result awesome re...|res awe res see o...|result awesome re...|
|seems small perfe...|     see sma per tra|seem smal

### Structured Data

#### Outcome Variable (Y): Customer Engagement Level

In [0]:
from pyspark.sql.functions import datediff, current_date
from pyspark.sql.functions import min, max
from pyspark.sql.functions import countDistinct
from pyspark.ml.feature import StringIndexer

In [0]:
#Encode event_type to map engagement levels
cosmetic_df = cosmetic_df.drop("event_type_index")

indexer = StringIndexer(inputCol="event_type", outputCol="event_type_index")
cosmetic_df = indexer.fit(cosmetic_df).transform(cosmetic_df)

cosmetic_df.select("event_type", "event_type_index").distinct().show()

+----------------+----------------+
|      event_type|event_type_index|
+----------------+----------------+
|        purchase|             3.0|
|            cart|             1.0|
|            view|             0.0|
|remove_from_cart|             2.0|
+----------------+----------------+



In [0]:
# Distribution of classes
cosmetic_df.groupBy("event_type_index").count().show()

+----------------+-------+
|event_type_index|  count|
+----------------+-------+
|             0.0|2032076|
|             1.0|1126995|
|             3.0| 263570|
|             2.0| 609461|
+----------------+-------+



#### Predictor Variables (X): Structured Features

In [0]:
from pyspark.sql.functions import datediff, current_date
from pyspark.sql.functions import min, max
from pyspark.sql.functions import countDistinct

In [0]:
#Recency: Days since the last interaction

cosmetic_df = cosmetic_df.withColumn("recency", datediff(current_date(), col("event_time")))

In [0]:
# Frequency: Total interactions per user session

frequency_df = cosmetic_df.groupBy("user_session").count().withColumnRenamed("count", "frequency")
cosmetic_df = cosmetic_df.join(frequency_df, on="user_session", how="left")

In [0]:
# Product Popularity: Number of purchases per product

product_popularity = cosmetic_df.filter(col("event_type") == "purchase").groupBy("cosmeticProductId").count().withColumnRenamed("count", "popularity")
cosmetic_df = cosmetic_df.join(product_popularity, on="cosmeticProductId", how="left")

In [0]:
#Session Diversity: Number of unique product categories viewed in a session

session_diversity = cosmetic_df.groupBy("user_session").agg(countDistinct("category_code").alias("session_diversity"))
cosmetic_df = cosmetic_df.join(session_diversity, on="user_session", how="left")

## Data Transformation

In [0]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.sql.functions import mean, stddev, col
from pyspark.sql.types import DoubleType, IntegerType

### Transform Cosmetic Data (Structured Interaction Data)

In [0]:
# Scale `cosmetic_price` column
assembler = VectorAssembler(inputCols=["cosmetic_price"], outputCol="price_vec")
cosmetic_df = assembler.transform(cosmetic_df)

scaler = StandardScaler(inputCol="price_vec", outputCol="price_scaled")
cosmetic_df = scaler.fit(cosmetic_df).transform(cosmetic_df)

### Transform Reviews Data with GPT Tokenizer (Unstructured Sentiment Data)

In [0]:
from transformers import AutoTokenizer, AutoModel

# Load a pre-trained GPT-like model
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
model = AutoModel.from_pretrained("EleutherAI/gpt-neo-1.3B")

# Encode text to embeddings
text = "This moisturizer is amazing for dry skin!"
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)
embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
print(embeddings)

2024-11-24 02:24:23.294516: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-24 02:24:23.350084: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-24 02:24:23.580436: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-24 02:24:25.916559: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[[-0.16730331 -1.3368105  -0.01049148 ...  0.62573016  1.1977668
  -1.2075031 ]]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [0]:
# Cast numerical columns to appropriate types
reviews_df = reviews_df.withColumn("mrp", reviews_df["mrp"].cast(DoubleType())) \
    .withColumn("review_price", reviews_df["review_price"].cast(DoubleType())) \
    .withColumn("product_rating", reviews_df["product_rating"].cast(DoubleType())) \
    .withColumn("product_rating_count", reviews_df["product_rating_count"].cast(IntegerType()))

### Transform Mapping Data

In [0]:
# Cast columns to IntegerType in `mapping_df`
mapping_df = mapping_df.withColumn("cosmeticProductId", mapping_df["cosmeticProductId"].cast(IntegerType()))
mapping_df = mapping_df.withColumn("reviewProductId", mapping_df["reviewProductId"].cast(IntegerType()))

# Calculate mean and standard deviation for filtering outliers
stats = mapping_df.select(
    mean("reviewProductId").alias("mean_reviews"),
    stddev("reviewProductId").alias("stddev_reviews")
).collect()[0]

mean_reviews = stats["mean_reviews"]
stddev_reviews = stats["stddev_reviews"]

# Filter out outliers beyond 3 standard deviations
mapping_df = mapping_df.filter(
    (col("reviewProductId") > mean_reviews - 3 * stddev_reviews) &
    (col("reviewProductId") < mean_reviews + 3 * stddev_reviews)
)

# Scale `cosmeticProductId` and `reviewProductId` in `mapping_df`
assembler = VectorAssembler(inputCols=["cosmeticProductId", "reviewProductId"], outputCol="mapping_features_vec")
mapping_df = assembler.transform(mapping_df)

scaler = StandardScaler(inputCol="mapping_features_vec", outputCol="mapping_scaled_features")
mapping_df = scaler.fit(mapping_df).transform(mapping_df)